# 3. Agentic RAG
<img src="https://humanloop.com/blog/rag-architectures/agentic-rag.png" height=250px>

In [2]:
%pip install deeplake==3.9.27 langchain-community==0.3.5 langchain-core==0.3.15 langchain-experimental==0.3.3 langchain-openai langchain-text-splitters==0.3.2 numpy==1.26.4 openai==1.54.4 pandas==2.2.3 pillow==10.4.0 PyMuPDF==1.24.13 tavily-python==0.5.0

  Using cached langchain_community-0.3.5-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_core-0.3.15-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached openai-1.54.4-py3-none-any.whl.metadata (24 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.3.11-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain-0.3.10-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain-0.3.9-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain-0.3.8-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain_community-0.3.5-py3-none-any.whl (2.4 MB)
Using cached langchain_core-0.3.15-py3-none-any.whl (408 kB)
Using cached langchain_text_splitters-0.3.2-py3-none-any.whl (25 kB)
Using cached openai-1.54.4-py3-none

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
graphrag 1.0.0 requires openai<2.0.0,>=1.57.0, but you have openai 1.54.4 which is incompatible.


### Importing all libraries

In [3]:
import os
from langchain_core.prompts import (
  ChatPromptTemplate,
)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools.retriever import create_retriever_tool
from langchain.tools.tavily_search import TavilySearchResults
from langchain.vectorstores.deeplake import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.memory import ConversationBufferWindowMemory, SQLChatMessageHistory
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.1.0) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setting up everything

In [4]:
tavily_api_key=""
os.environ["TAVILY_API_KEY"] = tavily_api_key
search_tool = TavilySearchResults(
  name="tavily_search_engine",
  description="A search engine optimized for retrieving information from web based on user query.",
  max_results=5,
  search_depth="advanced",
  include_answer=True,
  include_raw_content=True,
  include_images=True,
  verbose=False,
  tavily_api_key=tavily_api_key
)

In [5]:
from langchain_groq import ChatGroq
groq_api_key = ""
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

prompt = ChatPromptTemplate([
  ("system",
  f"""You are a helpful chatbot. You need to answer the user's queries in 
  detail from the document context. You have access to two tools: 
  deeplake_vectorstore_retriever and tavily_search_engine. 
  Always use the deeplake_vectorstore_retriever tool first to retrieve 
  the context and answer the question. If the context does not contain relevant 
  answer for the user's query, use the tavily_search_engine to fetch web search 
  results to answer them. NEVER give an incomplete answer. Always try your best 
  to find answer through web search if answer is not found from context."""),
  ("human", "{user_input}"),
  ("placeholder", "{messages}"),
  ("placeholder", "{agent_scratchpad}"),
])


### extracting and processing text from pdf

In [6]:
data = "pdf/IPCC_AR6_SYR_SPM.pdf" 
loader = PyMuPDFLoader(file_path=data) 
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200) 
docs = loader.load_and_split(text_splitter=text_splitter) 
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# embeddings = OpenAIEmbeddings(api_key="openai_api_key",) 
vectorstore = DeepLake(dataset_path="dataset", embedding=embeddings,) 
_ = vectorstore.add_documents(documents=docs)

c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document pdf/IPCC_AR6_SYR_SPM.pdf
  warnings.warn(
c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 5 of document pdf/IPCC_AR6_SYR_SPM.pdf
  warnings.warn(
c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 7 of document pdf/IPCC_AR6_SYR_SPM.pdf
  warnings.warn(


Deep Lake Dataset in dataset already exists, loading from the storage


c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 40 of document pdf/IPCC_AR6_SYR_SPM.pdf
  warnings.warn(
c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 41 of document pdf/IPCC_AR6_SYR_SPM.pdf
  warnings.warn(
Creating 193 embeddings in 1 batches of size 193:: 100%|██████████| 1/1 [00:13<00:00, 13.87s/it]

Dataset(path='dataset', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (386, 768)  float32   None   
    id        text      (386, 1)     str     None   
 metadata     json      (386, 1)     str     None   
   text       text      (386, 1)     str     None   


In [7]:
retriever_tool = create_retriever_tool(
	retriever=vectorstore.as_retriever(
	search_type="similarity",
	search_kwargs={
		"k": 6,
		"fetch_k": 12,
	},
),
name="deeplake_vectorstore_retriever",
  description="Searches and returns contents from uploaded Climate Change 2023 Sythesis Report based on user query.",
)

tools = [
  retriever_tool,
  search_tool,
]

In [8]:
history = SQLChatMessageHistory(
  session_id="ghdcfhdxgfx",
  connection_string="sqlite:///chat_history.db",
  table_name="message_store",
  session_id_field_name="session_id",
)

memory = ConversationBufferWindowMemory(chat_memory=history)

C:\Users\asus\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\asus\AppData\Local\Temp\ipykernel_30044\198912400.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(chat_memory=history)


In [9]:
agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
  tools=tools,
  agent=agent,
  verbose=False,
  max_iterations=8,
  early_stopping_method="force",
  memory=memory,
)

In [10]:
result = agent_executor.invoke({
  "user_input": "What are some climate related risks?",
},)
print(result["output"])

Here is the output:

Climate related risks are created by a range of hazards. Some are slow in their onset (such as changes in temperature and precipitation leading to droughts, or agricultural losses), while others happen more suddenly (such as tropical storms and floods). It is now widely recognised that climate-related impacts are not just a future threat.

Climate change-related risks to health include direct risks from extreme weather such as cold waves, storms, or prolonged high temperatures. There are also indirect risks such as mental health impacts of undernutrition or other environmental factors.

Climate-related risks may be , and their effects on the financial system subject to substantial uncertainty and tail-risk. The effects are spread over long time horizons, and actions today may determine the severity of risks in the years ahead.

As climate-related risks represent a financial risk, central banks and supervisors need to ensure that the financial system is resilient to

### Testing on questions which are not clearly present on docs

In [11]:
result = agent_executor.invoke({
  "user_input": "Who are Policymakers?",
},)
print(result["output"])

Policymakers are individuals who have the authority to make decisions and implement policies for a government, organization, or institution. In the context of climate change, policymakers play a crucial role in developing and implementing policies to mitigate and adapt to the impacts of climate change.

Examples of policymakers who are working on climate change include:

* Government officials, such as ministers, secretaries, and members of parliament, who are responsible for developing and implementing national and international climate policies.
* Energy and environmental regulators, who are responsible for developing and enforcing regulations to reduce greenhouse gas emissions and promote sustainable energy development.
* Members of international organizations, such as the United Nations, who are working to develop global agreements and frameworks to address climate change.
* Researchers and scientists, who are providing data and information to policymakers to inform their decisions